Prerequisites for colab. skip if you are running this notebook locally and are installing dependencies manually.
For more haystack installation options see https://docs.haystack.deepset.ai/docs/installation#installing-haystack-core

In [1]:
%%bash

pip install --upgrade pip
pip install datasets
pip install farm-haystack[colab]
pip install --upgrade haystack-memory
pip install --upgrade haystack-human-tool


UsageError: %%bash is a cell magic, but the cell body is empty.


In [1]:
# Activate logging
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [2]:
from getpass import getpass

openai_api_key = getpass("Enter OpenAI API key:")

Enter OpenAI API key:········


In [3]:
from haystack.document_stores import InMemoryDocumentStore, ElasticsearchDocumentStore

document_store = InMemoryDocumentStore(
    similarity="cosine",
    index="document",
    embedding_field="embedding",
    embedding_dim=1536
)


/opt/homebrew/Caskroom/miniforge/base/envs/haystack2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/rjtannous/github/vue-test/haystack/haystack/document_stores/search_engine.py:169: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  return self.client.indices.exists(index_name, headers=headers)
/Users/rjtannous/github/vue-test/haystack/haystack/document_stores/elasticsearch.py:499: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  indice

In [4]:
from datasets import load_dataset

# load seven wonders dataset from datasets and write documents into elasticsearch document store

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
document_store.write_documents(dataset)


In [4]:
from haystack.nodes import EmbeddingRetriever, OpenAIAnswerGenerator

# define retriever model , update dense vector embeddings on document store index and define generator model
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="text-embedding-ada-002",
    api_key=openai_api_key,
    max_seq_len=1024,
    top_k=4,
)

document_store.update_embeddings(retriever=retriever)

INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model text-embedding-ada-002


In [5]:
from haystack.nodes import PromptNode, PromptTemplate
from haystack.nodes import AnswerParser

QA_promptnode = PromptTemplate(
            name="zero-shot-QA", 
            prompt_text="You are a helpful and knowledgeable agent. Only Answer if the {documents} contain the answer. If the user question is not related to the provided {documents}, say I don't have an answer\n"
            "Question: {query}\n"
            "Answer:"
        )



In [6]:
QA_builder = PromptNode(model_name_or_path="text-davinci-003", api_key=openai_api_key, default_prompt_template=QA_promptnode)

In [7]:
from haystack import Pipeline

pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=QA_builder, name="Generator", inputs=["Retriever"])

In [9]:
from haystack.agents import Agent, Tool
from haystack.nodes import PromptNode
from haystack_human_tool.prompt_templates import agent_template

# create agent prompt node and define our Agent "memory_agent"
prompt_node = PromptNode(model_name_or_path="text-davinci-003", api_key=openai_api_key, max_length=512,
                         stop_words=["Observation:"])
memory_agent = Agent(prompt_node=prompt_node, prompt_template=agent_template)
# Define the first tool: A document store QA tool based on the pipeline
search_tool = Tool(name="DocumentStore_QA",
                   pipeline_or_node=pipe,
                   description="Access this tool to find information needed to answer questions.",
                   output_variable="results")

In [10]:
from haystack_memory.memory import MemoryRecallNode

sensory_memory = []
working_memory = []
memory_node = MemoryRecallNode(memory=working_memory)
memory_tool = Tool(name="Memory",
                   pipeline_or_node=memory_node,
                   description="Your memory. Always access this tool first to remember what you have learned.")

In [11]:
from haystack_human_tool.humaninloop import HumanInLoopNode

human_node = HumanInLoopNode(sensory_memory=sensory_memory)
human_tool = Tool(name="Human",
                  pipeline_or_node=human_node,
                  description="Access this tool to refine your query. Ask the human to rephrase the question or to explain what or who the human is referring to. Use the human's answer to rephrase your query for use as input to other tools")


In [12]:
# Add the human, memory and search tools to the agent
memory_agent.add_tool(memory_tool)
memory_agent.add_tool(human_tool)
memory_agent.add_tool(search_tool)


In [13]:
from haystack_memory.utils import MemoryUtils

# Initialize the agent with memory
memory_utils = MemoryUtils(working_memory=working_memory, sensory_memory=sensory_memory, agent=memory_agent)

In [14]:
#chat with agent example one
# enter zeus' golden sandals when prompted by agent
result = memory_utils.chat("where did his golden sandals rest?")


Agent Assistant started with {'query': 'where did his golden sandals rest?', 'params': None}
 find out what the person is talking about.
Tool: Human
Tool Input: Could you please explain what you mean by 'golden sandals'?
Could you please explain what you mean by 'golden sandals'?
zeus' golden sandals
Observation: zeus' golden sandals
Thought:  Let's use DocumentStore_QA to see if we can find any information about Zeus' golden sandals
Tool: DocumentStore_QA 
Tool Input: Where did Zeus' golden sandals rest?


Calculating embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.35it/s]
/Users/rjtannous/github/vue-test/haystack/haystack/document_stores/elasticsearch.py:376: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = self.client.search(index=index, body=body, request_timeout=300, headers=headers)["hits"]["hits"]


Observation: Zeus' golden sandals rested upon a footstool decorated with an Amazonomachy in relief. The passage underneath the throne was restricted by painted screens.
Thought:  This observation provides us with the answer to our question.
Final Answer: Zeus' golden sandals rested upon a footstool decorated with an Amazonomachy in relief. The passage underneath the throne was restricted by painted screens.

In [15]:
print(working_memory)

['where did his golden sandals rest?', "Could you please explain what you mean by 'golden sandals'?", "zeus' golden sandals", "Zeus' golden sandals rested upon a footstool decorated with an Amazonomachy in relief. The passage underneath the throne was restricted by painted screens."]


In [16]:
result = memory_utils.chat("which muslim general raided the city of Rhodes?")


Agent Assistant started with {'query': 'which muslim general raided the city of Rhodes?', 'params': None}
 access my memory to see if I can remember the answer. 
Tool: Memory
Tool Input: Which muslim general raided the city of Rhodes?
Observation: ['where did his golden sandals rest?', "Could you please explain what you mean by 'golden sandals'?", "zeus' golden sandals", "Zeus' golden sandals rested upon a footstool decorated with an Amazonomachy in relief. The passage underneath the throne was restricted by painted screens."]
Thought:  The observation I received from the memory tool does not contain the answer I am looking for. Let's try to refine my query and pass it to the DocumentStore_QA tool.
Tool: DocumentStore_QA
Tool Input: Who was the muslim general who raided the city of Rhodes?



Calculating embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.30it/s]
/Users/rjtannous/github/vue-test/haystack/haystack/document_stores/elasticsearch.py:376: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = self.client.search(index=index, body=body, request_timeout=300, headers=headers)["hits"]["hits"]


Observation: Muawiyah I was the Muslim general who raided the city of Rhodes in 653.
Thought:  This observation contains the answer I am looking for.

Final Answer: Muawiyah I was the Muslim general who raided the city of Rhodes in 653.

In [17]:
print(working_memory)

['where did his golden sandals rest?', "Could you please explain what you mean by 'golden sandals'?", "zeus' golden sandals", "Zeus' golden sandals rested upon a footstool decorated with an Amazonomachy in relief. The passage underneath the throne was restricted by painted screens.", 'which muslim general raided the city of Rhodes?', 'Muawiyah I was the Muslim general who raided the city of Rhodes in 653.']
